In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, username TEXT, password TEXT)')
cur.execute('DELETE FROM users')
cur.execute("INSERT INTO users (username, password) VALUES ('alice', 'wonderland')")
conn.commit()
conn.close()
print('example.db created with one user (alice)')


SQL Injection

In [2]:
# vulnerable_sql.py (DO NOT run against production DB)
import sqlite3

conn = sqlite3.connect('example.db')
cur = conn.cursor()

# imagine these values came from a web form
username = input("username: ")
password = input("password: ")

# RISKY: concatenating user input into SQL string
query = "SELECT id FROM users WHERE username = '" + username + "' AND password = '" + password + "';"
cur.execute(query)
row = cur.fetchone()
if row:
    print("Login success")
else:
    print("Login failed")

conn.close()


Login success


In [3]:
# safe_sql.py
import sqlite3

conn = sqlite3.connect('example.db')
cur = conn.cursor()

username = input("username: ")
password = input("password: ")

# SAFE: use parameterized queries - user data never becomes SQL code
cur.execute("SELECT id FROM users WHERE username = ? AND password = ?", (username, password))
row = cur.fetchone()
if row:
    print("Login success")
else:
    print("Login failed")

conn.close()


Login failed


Cross Site Scripting

In [11]:
# vulnerable_xss.py
from flask import Flask, request, render_template_string

app = Flask(__name__)

# A very small template that directly inserts the user comment into the page
TEMPLATE = """
<!doctype html>
<html>
  <head><title>Vulnerable XSS demo</title></head>
  <body>
    <h1>Leave a comment</h1>
    <form method="post" action="/comment">
      <input name="comment" placeholder="Type comment">
      <button type="submit">Submit</button>
    </form>

    <h2>Last comment</h2>
    <!-- UNSAFE: inserting user input directly -->
    <div id="last-comment">{{ comment | safe }}</div>
  </body>
</html>
"""

@app.route("/comment", methods=["GET", "POST"])
def comment():
    comment = ""
    if request.method == "POST":
        comment = request.form.get("comment", "")
    # render_template_string will insert the value of `comment` using Jinja2
    # (in this demo we are simulating a common mistake: showing user input
    # directly in a template without thinking about escaping)
    return render_template_string(TEMPLATE, comment=comment)

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Sep/2025 10:08:56] "POST /comment HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2025 10:09:19] "POST /comment HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2025 10:09:28] "POST /comment HTTP/1.1" 200 -


In [ ]:
# vulnerable_xss.py
from flask import Flask, request, render_template_string

app = Flask(__name__)

# A very small template that directly inserts the user comment into the page
TEMPLATE = """
<!doctype html>
<html>
  <head><title>Vulnerable XSS demo</title></head>
  <body>
    <h1>Leave a comment</h1>
    <form method="post" action="/comment">
      <input name="comment" placeholder="Type comment">
      <button type="submit">Submit</button>
    </form>

    <h2>Last comment</h2>
    <!-- UNSAFE: inserting user input directly -->
    <div id="last-comment">{{ comment }}</div>
  </body>
</html>
"""

@app.route("/comment", methods=["GET", "POST"])
def comment():
    comment = ""
    if request.method == "POST":
        comment = request.form.get("comment", "")
    # render_template_string will insert the value of `comment` using Jinja2
    # (in this demo we are simulating a common mistake: showing user input
    # directly in a template without thinking about escaping)
    return render_template_string(TEMPLATE, comment=comment)

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)


Shell Injection

In [ ]:
import os

host = input("Enter host to ping: ")

# RISKY: passing user input into shell command (shell interprets metacharacters)
os.system("ping -n 4 " + host)

1

In [13]:
# safe_cmd.py
import subprocess
import re

host = input("Enter host to ping: ")

# Simple validation: allow only letters, numbers, dots and hyphens (example whitelist)
if not re.fullmatch(r"[A-Za-z0-9\.-]+", host):
    raise ValueError("Invalid host")

# SAFE: pass argument list and avoid shell=True
subprocess.run(["ping", "-c", "4", host], check=True)


ValueError: Invalid host